In [21]:
import pandas as pd
import numpy as np
from scipy.stats import bernoulli, binom, norm, poisson,\
nbinom, geom

### 특수한 이산형 확률분포

- 베르누이 분포

In [3]:
# 기댓값과 분산 계산하기
p = 0.3
q = 1-p
E = p
V = p*q # 분산
print(f'[수기] 해당 분포 기댓값은 {E:.3f}, 분산은 {V:.3f}')

E, V = bernoulli.stats(p)
print(f'[라이브러리] 해당 분포의 기댓값은 {E:.3f} 분산은 {V:.3f}')

[수기] 해당 분포 기댓값은 0.300, 분산은 0.210
[라이브러리] 해당 분포의 기댓값은 0.300 분산은 0.210


- 이항분포

In [10]:
# combination 함수 정의하기
fac = np.math.factorial
def combi(a, b) : 
    result = fac(a)/fac(b)/fac(a-b)
    return result

# 근로자가 내년에 회사에서 일하지 않을 확률이 0.1이라고 추정한 경우,
# 시간제 근로자 3명을 무작위로 뽑았을 때 그 중 한 명이 금년에 회사를 떠날 확률은?
n = 3  # 샘플 개수
x = 1  # 확률 변수(떠날 샘플 개수)
p = 0.1

# 기댓값과 분산 계산하기
E = n*p
v = n*p*(1-p)
print(f'[수기] 해당 분포의 기댓값은 {E:.3f} 분산은 {V:.3f}')
E2, V2 = binom.stats(n, p)
print(f'[라이브러리] 해당 분포의 기댓값은 {E2:.3f} 분산은 {V2:.3f}', '\n')

# 확률질량함수 계산하기
pmf = combi(n, x) * p**x * (1-p)**(n-x)
print('[수기] 해당 분포의 확률질량함수(pmf): %.3f' % (pmf))
print('[라이브러리] 해당 분포의 확률질량함수(pmf): ', binom.pmf(x, n, p), '\n')

# 누적확률질량함수 계산하기: 1명 이하로 떠날 확률은?
cdf = 0
for i in range(0, x+1) : 
    cdf += binom.pmf(i, n, p)
print('[수기] 해당 분포의 누적확률질량함수(cdf): ', cdf)
print('[라이브러리] 해당 분포의 누적확률질럄함수(cdf): ', binom.cdf(x, n, p))

[수기] 해당 분포의 기댓값은 0.300 분산은 0.210
[라이브러리] 해당 분포의 기댓값은 0.300 분산은 0.270 

[수기] 해당 분포의 확률질량함수(pmf): 0.243
[라이브러리] 해당 분포의 확률질량함수(pmf):  0.243 

[수기] 해당 분포의 누적확률질량함수(cdf):  0.9720000000000001
[라이브러리] 해당 분포의 누적확률질럄함수(cdf):  0.972


In [14]:
# 이항분포의 정규분포 근사
x = 10
n = 30
p = 0.5

b_result = binom.pmf(x, n, p)
E = n*p
sigma = np.sqrt(E*(1-p))
zstat1 = (x-0.5-E)/sigma # 이항분포의 x에서 -0.5
zstat2 = (x+0.5-E)/sigma # 이항분포의 x에서 +0.5
n_result = norm.cdf(zstat2) - norm.cdf(zstat1)

print(f'이항분포로 계산하면 {b_result:.3f} 정규분포로 계산하면 {n_result:.3f}')

# 이항분포의 포아송분포 근사
x = 10
n = 100
p = 0.05
E = n*p

b_result = binom.pmf(x, n, p)
p_result = poisson.pmf(x, E)
print(f'이항분포로 계산하면 {b_result:.3f} 포아송분포로 근사계산하면 {p_result:.3f}')

이항분포로 계산하면 0.028 정규분포로 계산하면 0.028
이항분포로 계산하면 0.017 포아송분포로 근사계산하면 0.018


- 음이항분포

In [20]:
# 음이항분포
# A가 승리할 확률이 0.3이라면 5번째 경기에서 2번째로 이길 확률은?
fac = np.math.factorial
def combi(a, b) : 
    result = fac(a)/fac(b)/fac(a-b)
    return result

n = 5
k = 2
p = 0.3
q = 1-p

case = 2
if case == 1 :# 확률변수 X가 k번 성공할 때까지의 시행횟수가 x인 경우
    x = n
    E = k/p
    V = k*q/p**2
    pmf = combi(x-1, k-1) * p**k * q**(x-k)
    E2, V2 = np.nan, np.nan
    pkg_pmf, pkg_cdf = np.nan, np.nan
    cdf = 0
    for i in range(k, x+1) : 
        cdf += combi(i-1, k-1) * p**k * q**(i-k)
elif case == 2 : # 확률변수 X가 k번 성공할 때까지의 실패횟수 x인 경우
    x = n-k
    E = k*q / p
    V = k*q / p**2
    pmf = combi(x+k-1, k-1) * p**k * q**x
    E2, V2 = nbinom.stats(k, p)
    pkg_pmf, pkg_cdf = nbinom.pmf(x, k, p), nbinom.cdf(x, k, p)
    cdf = 0
    for i in range(0, x+1) : 
        cdf += nbinom.pmf(i, k, p)

# 기댓값과 분산 계산하기
print(f'[수기] 해당 분포의 기댓값은 {E:.3f} 분산은 {V:.3f}')
print(f'[라이브러리] 해당 분포의 기댓값은 {E2:.3f} 분산은 {V2:.3f}')

# 확률질량함수 계산하기
print('[수기] 확률질량함수(pmf): ', pmf)
print('[라이브러리] 확률질량함수(pmf): ', pkg_pmf, '\n')

# 누적확률질량함수 계산하기: 2번째 이하로 이길 확률은?
print('[수기] 누적확률질량함수(cdf): ', cdf)
print('[라이브러리] 누적확률질량함수(cdf): ', pkg_cdf)

[수기] 해당 분포의 기댓값은 4.667 분산은 15.556
[라이브러리] 해당 분포의 기댓값은 4.667 분산은 15.556
[수기] 확률질량함수(pmf):  0.12347999999999996
[라이브러리] 확률질량함수(pmf):  0.12348 

[수기] 누적확률질량함수(cdf):  0.47177999999999987
[라이브러리] 누적확률질량함수(cdf):  0.47178


-  기하분포

In [22]:
# 하나의 주사위를 세 번 던질 때 세 번째 시행에서 앞면 숫자가 6이 나올 확률?
n = 3
p = 1/6
q = 1-p

case = 1
if case == 1 : # 확률변수 X가 처음으로 성공할 때까지의 시행횟수 x인 경우
    x = n
    E = 1/p
    V = q/p**2
    pmf = q**(x-1)*p
    E2, V2 = geom.stats(p, moments='mv') # mean, variance
    pkg_pmf, pkg_cdf = geom.pmf(x, p), geom.cdf(x, p)
    cdf = 0
    for i in range(1, x+1) : 
        cdf += q**(i-1)*p
elif case == 2 : # 확률변수 X가 처음으로 성공할 때까지의 실패횟수 x인 경우
    x = n-1
    E = q/p
    V = q/p**2
    E2, V2 = np.nan, np.nan
    pmf = q**x * p
    pkg_pmf, pkg_cdf = np.nan, np.nan
    cdf = 0
    for i in range(0, x+1) : 
        cdf += q**i * p 

# 기댓값과 분산 계산하기
